In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

In [3]:
hr_employee = spark.read.csv("file:///home/hadoop/Downloads/HR_Employee.csv")

### Big DataFile Types
    Parquet FileFormat - records are column wise, this file format compress dataset of .csv or structured format into parquet format () parquet format will reduce file from original to compress by reducing filesize
    similar to this there are otheer file formats - orc, avro (stored records as keys and values, schema format is stored as JSON format)

In [6]:
#write csv file to local directory as parquet file format 
#hr_employee.write.parquet("file:///home/hadoop/Downloads/HR/")

In [7]:
#write csv file to local directory as parquet file format 
#hr_employee.write.orc("/HR_Data/")

In [8]:
!hdfs dfs -rm -r /HR_Data/

Deleted /HR_Data


In [9]:
hr_employee_parquet = spark.read.parquet("file:///home/hadoop/Downloads/HR/").show()

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+--------------------+---------------+--------------------+---------------+------------------+
|       _c0|                 _c1|                 _c2|      _c3|   _c4|_c5|          _c6|          _c7|           _c8|              _c9|          _c10|    _c11|           _c12|      _c13|  _c14|      _c15|    _c16|  _c17|                _c18|           _c19|                _c20|           _c21|              _c22|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+--------------------+---------------+--------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|A

#### Optimization techniques 
    optimizing queries significantly improve performance of spark running quries 
    spark jobs

#### 2. Partitioning 
    Divides datas into smaller chunks which can divide process datas into parallel 

In [10]:
partitioned_df = hr_employee.repartition(3)

In [12]:
#partitioned_df.write.parquet("/HR_Partition/")

#### 3. Caching & Persistance
    - Managing different level of storage

In [13]:
hr_employee.cache()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string]

In [14]:
# persistance of dataframe with specific storage level 

from pyspark import StorageLevel
hr_employee.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string]

#### 4. Serialization
    - Efficient Serialization Reduces time to read write data and transfer it over the network
    Kyro Serialization is popular methid for better performance over default java serialization 

##### a) Java serialization : It is default method. easy to use, draw back is it will slow down read write process. It can produce large serialized sizes.

In [15]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Java Serialization Example").getOrCreate()

##### b) Kyro Serialization : Faster

In [ ]:
"""
spark = SparkSession.builder \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrationRequired", "true") \
    .config("spark.kryo.classesToRegister", "org.apache.spark.example.anyClassname") \
    .appName("Kryo Serialization Example") \
    .getOrCreate()

"""

In [16]:
spark = SparkSession.builder.appName('PySparkSession')\
.config("spark.serializer","org.apache.spark.serializer.JavaSerializer").getOrCreate()

In [17]:
spark

#### 5. Broadcasting Variable
    * broadcasting small datasets improves join performance.

In [18]:
small_df = spark.read.csv("file:///home/hadoop/Downloads/airports(1).csv",inferSchema=True, header=True)
df = spark.read.csv("file:///home/hadoop/Downloads/raw_flight_data.csv", inferSchema=True, header=True)

In [19]:
from pyspark.sql.functions import broadcast

broadcast_df = broadcast(small_df)

In [24]:
broadcast_df.head()

Row(airport_id=10165, city='Adak Island', state='AK', name='Adak')

In [32]:
df.join(broadcast_df, df.OriginAirportID == broadcast_df.airport_id)

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int, airport_id: int, city: string, state: string, name: string]

In [33]:
df.printSchema()

root
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)



#### 6. Level of Parallelism

In [ ]:
#Adjust level based on cluster 
spark.conf.set("spark.default.parallelism",100)

#### 7. Avoid Groupby Key()

In [ ]:
# it increases number of shuffles

In [23]:
rdd = spark.sparkContext.parallelize([('dosa',2),('salad',3),('Idili',2),('appam',10)])

rdd.reduceByKey(lambda x,y:x+y).collect()

[('salad', 3), ('appam', 10), ('dosa', 2), ('Idili', 2)]

#### 8. Reduce Shuffle
    Reduce the number of shuffle by optimizing transformations
    use map() andd   reduce() over groupby
    use reduceByKe() over groupby
    

#### 9. Repartition with coalesce

#### 10.Accumulators
    -Aggregate info like count(),sum(),max(),std(),corr() etc.

In [26]:
acc = spark.sparkContext.accumulator(0)

In [28]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5,6,7,8,9])

In [29]:
def add(x):
    acc.add(x)

In [30]:
rdd.foreach(add)

In [31]:
print(acc.add)

45


#### 11.Bucketing
    - use to create Buckets of large datasets for effiecient query and joins

In [40]:
df.select("DayOfWeek").distinct().count()

7

In [42]:
df.write.bucketBy(7, "DayOfWeek").saveAsTable("bucketTable")

In [43]:
!hdfs dfs -cat /user/hive/warehouse/buckettable/part-00000-be8a4513-072b-4d1d-b90b-bd863740bc69_00001.c000.snappy.parquet

�nN �!�� Z! �RZ J ZD !%�> "#�NI  #2
!	i+^  "%+3	�^0 � 	�	�
O�� *FI � 	@� e 	~ �s!�p n BN:y  	��h �� ���:� 22�u U/B� �< N" J� �. 8>2" 2�� 2� ~k o� rA �r" b� 2�  INw� Yf��g V�RN
 j� 1u�+ND j jZ v.z v= �  �6� �v6 � 2N� ��S b. ��Rg R; �"� b� Rs � �
"�A��	� ^�J> f97�t J\ B�
  �+ R* 2IJ� �R4 VJ Z F�]V
  r	L#"�N) <� ��� �		2*N
�
Y� #�# ";0 1 "	 �(\3 12"!3 #"3"0!3�I#1D# 1#2<#"1""21133m !l02 3##1"s#"Y�1"# "��"32+8 1011 "e`30# 0!1!1 01#k2 �hP# 1 1 1"3!1	1!a##!e1#!o"##!01�11�"2y0!�3El###!~3#!-R3#1 �%��� 210#1 1##022!"-:" 1!k""2�(2"# 1"#0	�# 1�1�#"213QZ ["#0"��B �0"%{#�#
 #0A� 1!$W1"#!zAI A1] 1

�He �h�@�`�   @ (C � p>�g0x ���1� ��i`x(    xB�#`(���� 
 ���-�3h �$ � +�z �h @1�   �T �Y 	Rl �j
�%� �  � 0� h � "@ �Qx� �X@ � S� ��0��  aP	 d  �Q�j�1����   �2T X��P(�@L� ��@�� �@ �aB �4L 1��   �� ��� 3@�X   � L�@� <J  � P0 �u�T�S��Hp �Y�`  �b�"��(AA�b�C�p�F��Pp �BCP ���C�� �@@ ?�� �@F�(H( �	� ,#
!3R �T��f8@��  � -@�0D % a �.�� �T `
\ C�3P�(� .� u�D( �J�D �( x,!�A� �b�@ � 9

�� 
  ��P@    P��
� ) ) �CI(L	 7 &��4 3���QT�]�3hA  @0N�H�� 7�e�@�BQ F�i $2   ��a� $�i�� ��  `x&���c�  Q � : � � S p x<��Q8
�F��~������t��n�C ����C ����~����3�B�P~����P(
�@ �ۥR��@ ~���7��f'��   ��T*���b���t:!B=�P(�@  �@ b��@<��@ t:�N��f�T*E   �@     "�J�R�T*�x<�T*%R�T
�@ �X,C�R�T*��p8�@�j��@ R��@`0f0��P("�J�R�T*�J!B��h4� �@ -�J�8��@ �#��d���7H9�_*� R�T*F�!���p8)

B�Z-
q4
'�+��
�:��m� 
�����$�L��� 
7�!N�f>&�i� �& �� �&�"�
�H�9	�!�J�:�
W=�({ �! `>
4H�T&+�J��
-�!  ��Y���n )�a���)/�@�oQzKj�Ta=ш�K+�����Y�K�
�"�?�T��p�F0zt�Wk4�P( 	�c	��|bv(G�c	�
7����$��=T�r�o "�
��A@ !�+# �R�� ���!3��(D�r�\�t:�Rl ,!��R�����6�B��� ���l4�`�%F��
	� �������"��<?$�|�b0���p�|,v<��A �T
+0T�DAx<�[x�T*��qPGK�R)2x�^�V���l6���QD<��M&�	�V��r*U
�)��1�(<Z*�JA �����$�f�1y<��W �^$R�F��r*�
/�PH(��p8�M  �b����Ax�q/ eH*�J��hHj0E6$$D@`�p����{�^o,�>����p��ő ;�G�A ��ca| ��[TJ��e
$`0�`0@ ��d ��f�i�Z�0� F=%��p�V���O���Z�l6�H�heZ��h��b��y)T����d2�SHR(
�L�\.'�aqZuaj�Z'8V�L&��G�d2a�qT��l6�R"�Q�BF 
z �F"�(
 
%	�[,�L�	�4-�Z�!N�]$b�X&�c�X,Lc(Z�V��p8

�F� (<�� ���c�/D�����`�����@h
������6�����>4�Qh�>
��`4���I(>ڃ��%4�BCCyPh�����h�M�� l(����%�u�6�G���>�V�=>� ���Q>�O��(ڀAi�(��B�
퓢}><Z�� =h��֠U>���y��C�X0,
�h�y|(Z�A|F�A{$����^��A{(> ��C+�
��� <��|�T�g� h^��h�6�Q�5D�GC=�'���4-�4h��だ4x�GCAC���A�<
��x0����yT� �q�|��Fz<,푢5h���Ҡ)(/ڃVh0>��CC|����h<6M�Q�=F�ŧAi�,Ƈ��|���� \�ǣB��h�r�}h���`�/ˇ���
�x\������`|V�Aa|�Ƈ�yN�#E��h,�#x�4�A�4�ϋ�<4�Ԧ#0��@��h0�M���Ph�S��h� ��c��}�T��A�x��Ѡ=6ڣ£�(��x�>��NCs<h�G�Ӡ�n����=
���4h�G�$J聢4	����|
ͣ�0h�Oム�h�'��9>&��`�B
�A1R���x�Ap(x4ʃ�a���x4�O%�x�����H
�c�|>����=n�ƍ�=�AzX>��y|(���x �s�|.���8(��Sa���Ca|h���4����Q�[cD�Xh�O��P�
�G����x<JJА����I�>�Vx<h���i0*���!2��c���(J��`P>�Ãf�����V�|4 ��8��M�BcTh�����<h�G���5�#<(�ϣ��hnZば|d��a�(
�bK�Z���U� ��(XڧA�	>ŇҠ}�σHQH<
�� P��Aa<B���y,x���i<n��qP7��`P(<�G�x|(4����44�� �x����-xʇAk�$ܣ�x4�ŃG��h�O��h��C�)>���(P�

�ܰ��� �AT���l @�( �,������x��d�l��� 
<�4<����
K�p���Up �l1 �Np@�P0�8*8po8�C@Ѐ<�B �@ 
P"�@@ �P<�
3���0	+4 ���P�H��) �B!t�A@AA�L�@ 
 ��,@@  �
@  H`Dd� 8 ��P D��	;p0�/+t@(����@AF�C D427ԁ@�P@!�S�(@
��H8 � (B�B(@�2p�A0$
6��@ 4h��k��,��I<� 4d�l� 
��
(<��	А�- ����!KԱ�"� D�
���>����"'@F� 
,0A� 8�A'<��@;H { @�&���A0H%d� 6,�
�B����A`C�
 P@  ��+���@�<�A 0�	bt��@X��@9��JT�@`�A쀀@��Md�@	H�@ht� 
,0@,G3�D� 
��C 8� 6�� +@��B p@P��&ha`aD���	
(D�� �c�
t1A
MT2�T�� q@/l���r@�
 � 
p@�
5, (8� 8�(�'p0
��J�
���e-��`C�,p�@ ��� �$� XA ��<��p� #0 @�0�0@� �A$p @ 
<�!8��
8����0p 	p�� ��	�E����"��@ ��� �C@p@��A
p�A ��A �A ppE��p�@

8��<@ �� 
k �BP@ p�
4p �+Ѐ�� @
d����@ 8�  <����4p`� ��E��AY��@ >4�A6�� +@�
l�
+� B
�sb� xp��q�ep��l�HCt�� 6�0CPp@ 80���@i���
X!B
)dp@�C;l�@ @���8@  ����@�$w�
���<@ ��A$�@A��pC
#��@�@
 R�	DaC*Ԁ
����(@��Lr��d�
�Ap@@ppLp���B���PG(<0�  P�0�� 
*����p@ Tt�
؀
d� SP@�
BppB'<P��P� C,�@3���	$�  @ p�+h 
� ��H@EI �
����B P�@�� D�@�,�8 J � p� p�9t��L@ l�B �� � 4��D k,� 
$ pB t�B��bC�`
��
,��4�@2���_HF�]9��C\����I(p @ P@� ��@� 
E���  �!P��p� @ 'h�@P��(��'��@ �@Ad� �����p��8@�:8�  A l�@� � �$���v�B pЁ�@ �Ae�@ H� � 
�p0� ��<�8�
8`�#�S�< 
'��:C�@ #8��P���@ #H���P@
@ (8@�L�8�@ @�  PP@�P� p��<@ 
(� <��9�A@ �
$�D Յ�Sd0���@A @<�@
" @�8�pPZA��
�
l�A  �� P�

CP��P�� �(��C��fP <@� 1� �C�1
CLq��� 8� +eh� �8�p��C<������4A�4X��8�LaI p�B ,�W$AF � Ch0Ah� ���D	X @ ��l�
-8 "��[�Ќ4 d8@48�@�� D8@ � 
� (��
 @H���D���P� D� f p@0�@
 �A'!8��Cl0%�t���P��3P0   �Dl�@;�@4P� V�@�="P0�"l <�P�  l`_W`CP��;� Ix� d@�h@

@����p���s ��4�=H0�tpEdP  -p��ep�K�Є� �2O C �M<�B l��Hp�<@ p@ ����pBL�� �
p�I#�d�F��@9I�P@ B(2  <���Ѕ(8Ђ(� x0�ܠ�l1�� !� �p@���E �Xq�L0�	� (8� '6 H��H���H�D���� #��P p��Sd ���:� �� �  C<� 
!<�
d�A�pB'� (�2�#A @ t���	@��uH��MP0C�d���"A!8p��\�@ ',�@ ��@A�C�1dp� ��9A@',!C�P�@  @�3H @ ��VH� 84�l��
(p  0DLB� ���@� \��-a~l��5 p1�x���D��A� �<p IH�@)0��ē��BH��� DE8��l��$  �<�EL��3"L` �E ȐC4��6x �P

/0`�B-���t� ��� %�`�	�`�(�@�AA4��,�0 O�`�#� L� ��%L0B%�B&$a 7��@�H�
�`D4��\\ x#��@@D�Ѓ&�D#4@C&�`� &�@A&� ��!R{ ��4�P�;DX ?�AB1�@ 0Q���7 ����Ѐ&䠇-,�e��B/<@	5`��(.L�@=�B L�F?�Xq�x�	=�a@* �S 0���C,İ	'x �%@�!� ��A Xp�7�p�&��  �a�\P܌2�941E-�@D��AK@Bf�&��� ,`��(�0�I��N,� B6�0�\T� @C9�p����!� F� ��:�p�=�"��qDRt� k`�
� A$� @1X��?��� \ � �Ђ	(�B!� D,4sK �
���X��<	<��"4��R9 0�8Ȱ	g��1x��
0��*J�ЃK,�A%|��D��$�P@��1�(���2���/��E�`����	�@A-T@A�h�� ��u`�$(�I1L���*Q��2�ڠ�  a�41V�B H�� BH��q=d"�K  �
P � b��A98d�	��`�D�@	-,!E��k�!��0��� -�?���- �K4��4���X��<V���/,5B4<�D� ,"� 0� �dl�Ig4��*I��b�CD4U���� A��Q�4�@�8� #- � �@7\�E�
l� <�-��@ ,��	,<PB � �,���!�9�p�5�,��4< �/ @� �� ��8���-�@��i��E O@�Bx� JpaD*t �Sp��S��B�	-�c %�Q 4, �!��� ��� 
� �B)~А

k|�1�bDR��?���%��
44P�%�����@ pC�<l� �\3
&X��,` �P`�0A;2,��E���4�0�<�D,�@6L�B-�1�7��x@�-L@��
t0 
aF��Յ��#H�!8���lܲ�
, CQ @� 9<��%  &��:����,�B
`���\���� �1҃0��
�1X@
&@�?H1�Et� #���* |0B���
$1�9X�& ��x�D ��H=<�)�1�-��� E���Kd�4PF� ��@EIP���Jf@��4�p�@��q�j$�  tp���	4���?0�
Ȁ�
 �@DKc�,`�	,,�@�A D<@�(_��
<@ eX�CPP� � @F@ #4�0
	4<��%�a�	1<`G �@�P �
���XpLv���X0���//�у<#�3�/<A��@C��� ,P:��nH�@5,(L�?pG0��?�`�:x@CgP,����AC$BCd�0
�D,��BB��hT�&qC8�K��ð ,��`=��L�A	�� =t@��  l��#X� �-,� , ��`B�@�2<`Da9���3�@J, 	0X � 4p�;\�K���@&��@#�P��0 ���D���P#,�� �Q�����P�2, 
G���u�-���`� �� $���DhQ\����0A=  �	��AI|@A,���,$ *,�E+�Ђ,��,����( |`*prl�*Q��D<h�7@��D� �,q	
 ���4���@vL�� ��1��D?<0�=����BJ�@Dg��

-�  � 0���V p �@0	�@�t0�� ./	�4A`�(V��w�0� ,�0�I$���	#԰[��B�� j6$�1h#�A|$K  ��%,�C {��/ � 7�! N,,P�(�\P@��:/�A\0�� �k �:U��/h��)
$1E�|@�x��
3�Q/&x��1{�L,Vؔ�8�p�4�Hdr� &\P9 �
�cF9#��@� �0@P��PM (�����P�@M�>( @� 0A~�0�N@Ԃ
)���<�0�p	O���IX@/���
�CYx �kH�D�����A�b%�`��  C7�@	��1 �P �@��b���Q�`��UB <P�
*�p0���)��B	D�b (<`A��@A-S��
0p@:��
 t`GF< HBG<��DAC	r`�РC
�0A$��
���/Ch�P��-�`eT1E3��d<�FB�R��#����P��0�op��@�� �=|P�4�D=%$ф)o�BL���� -|�N;Ta�Ϙ�@Q���k�@B?���[��8��x@KTA�7D�BD-h�"�I�`�=�AJ!�PA	7<��E�(��QB/u� ��/8,р �`8�T$� D���\L��� A �<a=��G;B� � X�>%�Є4�@�?d��=x�� -4*�6��hq��04P��d�B$|P�-4P�4\P = �� /,�@'� (L����Dd��7\0� @�Af,�	KP�?���/أ�%�@4s�T�J����/�@1IP`�	#����0 
�� =� �&,A�4���BY C0 �';8�
�hr&h �B"�!4�Ў/ PB  �`
 �B8���,X`AQ�